In [17]:
%cd "/data/ai_club/team_C/"
! pip install keras_tuner

/data/ai_club/team_C
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [18]:
from sklearn import svm
import numpy as np
from sklearn.model_selection import train_test_split
import joblib
from spectral import *
from PIL import Image, ImageFile
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D 
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import keras_tuner as kt

In [19]:
#Model Setup
#Data Files
#
data_train = np.load('CNN/spectral_lymph_tile_array_3_9_22_left_update_1.npy')
non_data_train = np.load('CNN/spectral_non_tile_array_3_9_22_left_update_1.npy')
data_test = np.load('CNN/spectral_lymph_tile_array_3_9_22_right_update_1.npy')
non_data_test = np.load('CNN/spectral_non_tile_array_3_9_22_right_update_1.npy')

data_stack_train = np.vstack((data_train, non_data_train)) # One array of all training data
data_stack_test = np.vstack((data_test, non_data_test))   # One array of all test data
labels_train = np.append(np.ones(data_train.shape[0]), np.zeros(non_data_train.shape[0])) 
labels_test = np.append(np.ones(data_test.shape[0]), np.zeros(non_data_test.shape[0]))

X_train = data_stack_train[:, :]
y_train = labels_train[:]
X_test =  data_stack_test[:, :]
y_test = labels_test[:]
print("Data Loaded")


Data Loaded


In [20]:
def model_builder(hp):
    #Model
    model=Sequential()
    model.add(Conv2D(filters=128,kernel_size=3,padding="same",activation="relu",input_shape=(17,17,394)))
    model.add(Conv2D(filters=16,kernel_size=3,padding="same",activation="relu"))
    model.add(Flatten())
    hp_units = hp.Int('units', min_value=64, max_value=1024, step=64)
    model.add(Dense(units=hp_units, activation='relu'))
    hp_units2 = hp.Int('units', min_value=64, max_value=1024, step=64)
    model.add(Dense(units=hp_units2, activation='relu'))
    model.add(Dense(1,activation="sigmoid"))  
    print(model.summary())
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) #compile Model
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=optimizers.Adam(learning_rate=hp_learning_rate),
                loss='binary_crossentropy',
                metrics=['accuracy'])
    return model

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=50,
                     factor=3,
                     overwrite=True)
                     #directory='Lymphocyte_Classification/ML Experiments/',
                     #project_name='Lymphcytes')
                    
#stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(X_train,y_train, epochs=15, validation_data=(X_test,y_test))#, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1 )[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and The optimal number of units in the second densely-connected
layer is {best_hps.get('units')} the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 15 Complete [00h 01m 05s]
val_accuracy: 0.9173488020896912

Best val_accuracy So Far: 0.9353125691413879
Total elapsed time: 00h 16m 28s

Search: Running Trial #16

Value             |Best Value So Far |Hyperparameter
640               |640               |units
0.01              |0.0001            |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
3                 |3                 |tuner/bracket
0                 |0                 |tuner/round

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 17, 17, 128)       454016    
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 17, 17, 16)        18448     
_________________________________________________________________
flatten (Flatten)            (None, 4624)           